# Relationship 多对多


## 测试准备


### 定义 model


In [1]:
from typing import List, Optional

from sqlmodel import Field, Relationship, Session, SQLModel, create_engine, select


# 多对多表
class HeroTeamLink(SQLModel, table=True):
    team_id: Optional[int] = Field(
        default=None, foreign_key="team.id", primary_key=True
    )
    hero_id: Optional[int] = Field(
        default=None, foreign_key="hero.id", primary_key=True
    )


class Team(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    headquarters: str

    heroes: List["Hero"] = Relationship(back_populates="teams", link_model=HeroTeamLink)


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    secret_name: str
    age: Optional[int] = Field(default=None, index=True)

    team_id: Optional[int] = Field(default=None, foreign_key="team.id")
    teams: List[Team] = Relationship(back_populates="heroes", link_model=HeroTeamLink)

### 创建表


In [2]:
sqlite_file_name = "relationship_many_to_many_database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=False)
SQLModel.metadata.create_all(engine)

## 创建 Relationship


In [3]:
with Session(engine) as session:
    team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
    team_z_force = Team(name="Z-Force", headquarters="Sister Margaret's Bar")

    hero_deadpond = Hero(
        name="Deadpond",
        secret_name="Dive Wilson",
        teams=[team_z_force, team_preventers],
    )
    hero_rusty_man = Hero(
        name="Rusty-Man",
        secret_name="Tommy Sharp",
        age=48,
        teams=[team_preventers],
    )
    hero_spider_boy = Hero(
        name="Spider-Boy", secret_name="Pedro Parqueador", teams=[team_preventers]
    )
    session.add(hero_deadpond)
    session.add(hero_rusty_man)
    session.add(hero_spider_boy)
    session.commit()

    session.refresh(hero_deadpond)
    session.refresh(hero_rusty_man)
    session.refresh(hero_spider_boy)

    print("Deadpond:", hero_deadpond)
    print("Deadpond teams:", hero_deadpond.teams)
    print("Rusty-Man:", hero_rusty_man)
    print("Rusty-Man Teams:", hero_rusty_man.teams)
    print("Spider-Boy:", hero_spider_boy)
    print("Spider-Boy Teams:", hero_spider_boy.teams)

Deadpond: name='Deadpond' secret_name='Dive Wilson' team_id=None age=None id=1
Deadpond teams: [Team(name='Z-Force', headquarters="Sister Margaret's Bar", id=1), Team(name='Preventers', headquarters='Sharp Tower', id=2)]
Rusty-Man: name='Rusty-Man' secret_name='Tommy Sharp' team_id=None age=48 id=2
Rusty-Man Teams: [Team(name='Preventers', headquarters='Sharp Tower', id=2)]
Spider-Boy: name='Spider-Boy' secret_name='Pedro Parqueador' team_id=None age=None id=3
Spider-Boy Teams: [Team(name='Preventers', headquarters='Sharp Tower', id=2)]


## Update and Remove

In [4]:
with Session(engine) as session:
    hero_spider_boy = session.exec(select(Hero).where(Hero.name == "Spider-Boy")).one()
    team_z_force = session.exec(select(Team).where(Team.name == "Z-Force")).one()
    team_z_force.heroes.append(hero_spider_boy)
    session.add(team_z_force)
    session.commit()

    print("Spider-Boy's Teams:", hero_spider_boy.teams)
    print("Z-Force heroes:", team_z_force.heroes)

    hero_spider_boy.teams.remove(team_z_force)
    session.add(team_z_force)
    session.commit()
    print("Spider-Boy's teams:", hero_spider_boy.teams)
    print("Z-Force's heroes:", team_z_force.heroes)


Spider-Boy's Teams: [Team(name='Preventers', headquarters='Sharp Tower', id=2), Team(name='Z-Force', headquarters="Sister Margaret's Bar", id=1)]
Z-Force heroes: [Hero(name='Deadpond', secret_name='Dive Wilson', team_id=None, age=None, id=1), Hero(name='Spider-Boy', secret_name='Pedro Parqueador', team_id=None, id=3, age=None)]
Spider-Boy's teams: [Team(name='Preventers', headquarters='Sharp Tower', id=2)]
Z-Force's heroes: [Hero(name='Deadpond', secret_name='Dive Wilson', team_id=None, age=None, id=1)]
